In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/science-topic-classification/SampleSubmission.csv
/kaggle/input/science-topic-classification/train.csv
/kaggle/input/science-topic-classification/test.csv


In [31]:
import os
import re

import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, regularizers, constraints
from keras.layers import *
from keras.models import Model

In [5]:
## 데이터불러오기

train = pd.read_csv('/kaggle/input/science-topic-classification/train.csv',encoding='ISO-8859-1')
train_label = train[['label']]
train_data = train[['TITLE','ABSTRACT']]

In [6]:
test = pd.read_csv('/kaggle/input/science-topic-classification/test.csv',encoding='ISO-8859-1')
test_data = test[['TITLE','ABSTRACT']]


In [49]:
sub = pd.read_csv('/kaggle/input/science-topic-classification/SampleSubmission.csv')
sub

,ID,label
0,1,-1
1,2,-1
2,3,-1
3,4,-1
4,5,-1
...,...,...
4839,4840,-1
4840,4841,-1
4841,4842,-1
4842,4843,-1


# 전처리

In [7]:
shortword = re.compile(r'\W*\b\w{1,2}\b')
shortword.sub('', 'i am optimus prime')

' optimus prime'

In [8]:
## 숫자 제거
test_data.TITLE = test_data.TITLE.apply(lambda x: re.sub(r'[0-9]+', '',x))
test_data.ABSTRACT = test_data.ABSTRACT.apply(lambda x: re.sub(r'[0-9]+', '',x))
train_data.TITLE = train_data.TITLE.apply(lambda x: re.sub(r'[0-9]+', '',x))
train_data.ABSTRACT = train_data.ABSTRACT.apply(lambda x: re.sub(r'[0-9]+', '',x))

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [112]:
## 1~2단어 제거
one_two = re.compile(r'\W*\b\w{1,2}\b')
test_data.TITLE = test_data.TITLE.apply(lambda x: one_two.sub('', x))
test_data.ABSTRACT = test_data.ABSTRACT.apply(lambda x: one_two.sub('', x))
train_data.TITLE = train_data.TITLE.apply(lambda x:one_two.sub('', x))
train_data.ABSTRACT = train_data.ABSTRACT.apply(lambda x:one_two.sub('', x))

In [113]:
## 특수문자 제거
pattern_punctuation = re.compile(r'[^\w\s]')
test_data.TITLE = test_data.TITLE.apply(lambda x: pattern_punctuation.sub('', x))
test_data.ABSTRACT = test_data.ABSTRACT.apply(lambda x: pattern_punctuation.sub('', x))
train_data.TITLE = train_data.TITLE.apply(lambda x:pattern_punctuation.sub('', x))
train_data.ABSTRACT = train_data.ABSTRACT.apply(lambda x:pattern_punctuation.sub('', x))

# Tokenizer

In [114]:
total_data = pd.concat([train_data,test_data])
total_data

,TITLE,ABSTRACT
0,Detecting the impact public transit the transm...,many developing countries public transit play...
1,Proxima Centauri habitable study atmospheric ...,address the important question whether the ne...
2,Verifying Security Protocols using Dynamic Str...,Current formal approaches have been successf...
3,Scenic LanguageBased Scene Generation,Synthetic data has proved increasingly usefu...
4,NearOptimal Discrete Optimization for Experime...,The experimental design problem concerns the...
...,...,...
4839,Tight contact structures Seifert surface compl...,consider complements standard Seifert surface...
4840,Graded components Local cohomology modules,Let commutative Noetherian ring containing f...
4841,Levitation nonmagnetizable droplet inside ferr...,The central theme this work that stable levi...
4842,Helicity convective flows from localized heat ...,Experimental and numerical study the steadys...


In [116]:
from keras.preprocessing.text import Tokenizer
title_tk = Tokenizer()
abs_tk = Tokenizer()
title_tk.fit_on_texts(total_data['TITLE'])
abs_tk.fit_on_texts(total_data['ABSTRACT'])

# Text to Sequences

In [117]:
title_tseq = title_tk.texts_to_sequences(total_data['TITLE'])
abs_tseq = abs_tk.texts_to_sequences(total_data['ABSTRACT'])

In [118]:
len(title_tseq[0]), len(abs_tseq[0])

(8, 162)

# Pad Sequence

In [181]:
from keras.preprocessing.sequence import pad_sequences

title_tpad = pad_sequences(title_tseq, maxlen=30, padding= 'post')
abs_tpad = pad_sequences(abs_tseq, maxlen= 200,padding= 'post')

In [182]:
len(abs_tpad[0])

200

In [183]:
len(title_tpad[0])

30

# Train Test 분리

In [184]:
title_train = title_tpad[:15472]
abs_train = abs_tpad[:15472]

title_test = title_tpad[15472:]
abs_test = abs_tpad[15472:]

# label dummy 코딩

In [185]:
target = pd.get_dummies(train_label['label'])
target

,0,1,2,3
0,1,0,0,0
1,0,1,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
15467,0,0,1,0
15468,0,0,1,0
15469,0,1,0,0
15470,1,0,0,0


# Modeling

In [128]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from keras.models import Sequential, Model, Input

In [237]:
title_gru_input = Input(shape=(30,))
title_gru = Embedding(len(title_tk.word_index)+1,20,input_length=30)(title_gru_input)
title_gru = GRU(128,return_sequences=True)(title_gru)
#title_gru = Bidirectional(GRU(32, return_sequences=True))(title_gru)
title_gru = GlobalMaxPooling1D()(title_gru)

abs_gru_input = Input(shape=(200,))
abs_gru = Embedding(len(abs_tk.word_index)+1,200,input_length=200)(abs_gru_input)
abs_gru = GRU(128,return_sequences=True)(abs_gru)
#abs_gru = Bidirectional(GRU(32, return_sequences=True))(abs_gru)
abs_gru = GlobalMaxPooling1D()(abs_gru)

from keras.layers.merge import concatenate
model = concatenate([title_gru, abs_gru]) 

model = Dense(128, activation='relu')(model) 
model = Dropout(0.5)(model)
model = Dense(64, activation='relu')(model) 
model = Dropout(0.3)(model)
model = Dense(4, activation='sigmoid')(model)

ensemble_model = Model([title_gru_input,abs_gru_input], model)

ensemble_model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['acc'])

ensemble_model.fit([title_train, abs_train], target, epochs=1, batch_size=32, validation_split = 0.2)

In [239]:
ensemble_model.fit([title_train, abs_train], target, epochs=1, batch_size=32, validation_split = 0.2)

387/387 [==============================] - 10s 25ms/step - loss: 0.4035 - acc: 0.8574 - val_loss: 0.5273 - val_acc: 0.8158


In [188]:
pred4=ensemble_model.predict([title_test,abs_test])

In [202]:
pred6=ensemble_model.predict([title_test,abs_test])

In [287]:
## 앙상블 적용
ensemble2 = (pred4 + pred6+ pred2 + )/3

In [288]:
pred_list=[]
for i in ensemble2:
    pred_list.append(np.argmax(i))
    
sub['label'] = pred_list
sub.to_csv('ensemble_final_1.csv',index=False)